<div class="alert alert-block alert-success">
    <h1>
        Example notebook - Healthcare
    </h1>
    <p>
        Link to dataset : <a href="https://www.kaggle.com/datasets/prasad22/healthcare-dataset">Kaggle link</a>
    </p>
</div>

# Import modules and functions

In [1]:
import os
import pandas as pd
import re
import time
from tqdm.auto import tqdm

from turingdb_examples.utils import create_ID_column
from turingdb_examples.graph import (
    create_graph_from_df,
    networkx_to_jsonl
)
from turingdb_examples.llm import natural_language_to_cypher

In [2]:
%load_ext autoreload
%autoreload 2

# Check data files are available

In [3]:
example_name = "healthcare_dataset"
path_data = f"{os.getcwd()}/data/{example_name}"
if not os.path.exists(path_data):
    raise ValueError(f"{path_data} does not exists")

filename = "healthcare_dataset.csv"
list_csv_files = sorted(os.listdir(path_data))
if filename not in list_csv_files:
    raise ValueError(
        f"{filename} csv file is not available in {path_data}"
    )

# Import and format data

In [5]:
df = pd.read_csv(f"{path_data}/healthcare_dataset.csv")
df["Name"] = df["Name"].apply(
    lambda x: f"{x.split(' ')[0].capitalize()} {x.split(' ')[1].upper()}"
)
df["Doctor"] = df["Doctor"].apply(
    lambda x: f"{x.split(' ')[0].capitalize()} {x.split(' ')[1].upper()}"
)
df = create_ID_column(df)
# Keep only 10 patients to reduce graph for now
# You can comment the following line to generate the whole graph
#df = df.iloc[:5000, :]
df

,Patient ID,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,00000,Bobby JACKSON,30,Male,B-,Cancer,2024-01-31,Matthew SMITH,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,00001,Leslie TERRY,62,Male,A+,Obesity,2019-08-20,Samantha DAVIES,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,00002,Danny SMITH,76,Female,A-,Obesity,2022-09-22,Tiffany MITCHELL,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,00003,Andrew WATTS,28,Female,O+,Diabetes,2020-11-18,Kevin WELLS,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,00004,Adrienne BELL,43,Female,AB+,Cancer,2022-09-19,Kathleen HANNA,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55495,55495,Elizabeth JACKSON,42,Female,O+,Asthma,2020-08-16,Joshua JARVIS,Jones-Thompson,Blue Cross,2650.714952,417,Elective,2020-09-15,Penicillin,Abnormal
55496,55496,Kyle PEREZ,61,Female,AB-,Obesity,2020-01-23,Taylor SULLIVAN,Tucker-Moyer,Cigna,31457.797307,316,Elective,2020-02-01,Aspirin,Normal
55497,55497,Heather WANG,38,Female,B+,Hypertension,2020-07-13,Joe JACOBS,"and Mahoney Johnson Vasquez,",UnitedHealthcare,27620.764717,347,Urgent,2020-08-10,Ibuprofen,Abnormal
55498,55498,Jennifer JONES,43,Male,O-,Arthritis,2019-05-25,Kimberly CURRY,"Jackson Todd and Castro,",Medicare,32451.092358,321,Elective,2019-05-31,Ibuprofen,Abnormal


# Create graph from dataframe

In [6]:
label_str = "displayName"

G = create_graph_from_df(
    df,
    directed=True,
    source_node_col={"id": "Patient ID", label_str: "Name", "type": "Patient"},
    attributes_source_node_cols=["Age", "Date of Admission", "Discharge Date"],
    optional_nodes_cols={
        "Gender": {"link_to_source": True, "edge_type_to_source": "is"},
        "Blood Type": {"link_to_source": True, "edge_type_to_source": "is"},
        "Medical Condition": {"link_to_source": True, "edge_type_to_source": "has"},
        "Doctor": {"link_to_source": True, "edge_type_to_source": "is_treated_by"},
        "Hospital": {
            "attributes": ["Room Number"],
            "link_to_source": True,
            "edge_type_to_source": "is_treated_in",
        },
        "Insurance Provider": {
            "attributes": ["Billing Amount"],
            "link_to_source": True,
            "edge_type_to_source": "is_client_of",
        },
        "Admission Type": {"link_to_source": True},
        "Medication": {
            "link_to_source": True,
            "edge_type_to_source": "took_medication",
        },
        "Test Results": {"link_to_source": True, "edge_type_to_source": "has_result"},
    },
)
print(f"Resulting graph : {G}")

Resulting graph : DiGraph with 134984 nodes and 499500 edges


In [7]:
# Show first few nodes with properties
for node in list(G.nodes(data=True))[:20]:
    print(node)

('00000', {'displayName': 'Bobby JACKSON', 'type': 'Patient', 'Age': 30, 'Date of Admission': '2024-01-31', 'Discharge Date': '2024-02-02'})
('Male', {'displayName': 'Male', 'type': 'Gender'})
('B-', {'displayName': 'B-', 'type': 'Blood Type'})
('Cancer', {'displayName': 'Cancer', 'type': 'Medical Condition'})
('Matthew SMITH', {'displayName': 'Matthew SMITH', 'type': 'Doctor'})
('Sons and Miller', {'displayName': 'Sons and Miller', 'type': 'Hospital', 'Room Number': 328})
('Blue Cross', {'displayName': 'Blue Cross', 'type': 'Insurance Provider', 'Billing Amount': 18856.281305978155})
('Urgent', {'displayName': 'Urgent', 'type': 'Admission Type'})
('Paracetamol', {'displayName': 'Paracetamol', 'type': 'Medication'})
('Normal', {'displayName': 'Normal', 'type': 'Test Results'})
('00001', {'displayName': 'Leslie TERRY', 'type': 'Patient', 'Age': 62, 'Date of Admission': '2019-08-20', 'Discharge Date': '2019-08-26'})
('A+', {'displayName': 'A+', 'type': 'Blood Type'})
('Obesity', {'displa

In [8]:
# Show first few edge with properties
for edge in list(G.edges(data=True))[:20]:
    print(edge)

('00000', 'Male', {'type': 'is'})
('00000', 'B-', {'type': 'is'})
('00000', 'Cancer', {'type': 'has'})
('00000', 'Matthew SMITH', {'type': 'is_treated_by'})
('00000', 'Sons and Miller', {'type': 'is_treated_in'})
('00000', 'Blue Cross', {'type': 'is_client_of'})
('00000', 'Urgent', {})
('00000', 'Paracetamol', {'type': 'took_medication'})
('00000', 'Normal', {'type': 'has_result'})
('00001', 'Male', {'type': 'is'})
('00001', 'A+', {'type': 'is'})
('00001', 'Obesity', {'type': 'has'})
('00001', 'Samantha DAVIES', {'type': 'is_treated_by'})
('00001', 'Kim Inc', {'type': 'is_treated_in'})
('00001', 'Medicare', {'type': 'is_client_of'})
('00001', 'Emergency', {})
('00001', 'Ibuprofen', {'type': 'took_medication'})
('00001', 'Inconclusive', {'type': 'has_result'})
('00002', 'Female', {'type': 'is'})
('00002', 'A-', {'type': 'is'})


# Create `turingdb` python client

<div class="alert alert-block alert-info">
    <h2>
        See <a href="https://docs.turingdb.ai/quickstart">TuringDB Get started documentation</a> for the important steps to follow :
    </h2>
    <h3>
        Remember to have your <code>turingdb</code> server running while working in this notebook !
    </h3>
</div>

In [9]:
from turingdb import TuringDB

# Create TuringDB client
# set host parameter to the URL (as string) on which TuringDB is running,
# default "http://localhost:6666"
client = TuringDB(host="http://localhost:6666")
try:
    client.warmup()
except Exception as e:
    print(f"TuringDB not started, please run `uv run turingdb` in your terminal")

In [10]:
# Get list of available graphs
list_graphs = client.list_available_graphs()

In [11]:
# Get list of loaded graphs
client.list_loaded_graphs()

['crypto_orbitaal_fraud_detection1', 'default']

# Set graph name

In [12]:
# Set graph name
graph_name_prefix = example_name
graph_name_nb_suffix = str(
    max(
        [
            int(re.sub(graph_name_prefix, "", g))
            for g in list_graphs
            if g.startswith(graph_name_prefix)
            and re.sub(graph_name_prefix, "", g).isdigit()
        ]
        + [0]
    )
    + 1
)
graph_name = graph_name_prefix + graph_name_nb_suffix
graph_name = re.sub("-", "_", graph_name)
graph_name

'healthcare_dataset1'

# Create JSONL file

In [13]:
turingdb_dir = os.path.expanduser("~/.turing/data")
if not os.path.exists(turingdb_dir):
    raise ValueError(f"""
    TuringDB directory {turingdb_dir} does not exist.
    Make sure the value you set here is the same you set when running turingdb.
    """)

In [14]:
%%time

# Write graph to JSONL format
print("Writing graph to JSONL...")
jsonl_filename = networkx_to_jsonl(
    G,
    graph_name,
    node_type_key="type",
    edge_type_key="type",
    data_dir=turingdb_dir
)

Writing graph to JSONL...
JSONL file written to: /home/ubuntu/.turing/data/healthcare_dataset1.jsonl
Graph: 134,984 nodes, 499,500 edges
CPU times: user 1.44 s, sys: 0 ns, total: 1.44 s
Wall time: 1.44 s


# Create `turingdb` graph

In [15]:
print(f"Creating graph: {graph_name}")

Creating graph: healthcare_dataset1


In [16]:
%%time

start_time = time.time()

# Load JSONL into TuringDB
print(f"\nLoading JSONL into TuringDB as graph '{graph_name}' ...")
client.query(f"LOAD JSONL '{jsonl_filename}' AS {graph_name}")

execution_time = time.time() - start_time
print(f"\n\u2713 Graph '{graph_name}' created in {execution_time:.2f} seconds")

# Set active graph for querying
client.set_graph(graph_name)


Loading JSONL into TuringDB as graph 'healthcare_dataset1' ...

✓ Graph 'healthcare_dataset1' created in 1.42 seconds
CPU times: user 2.48 ms, sys: 0 ns, total: 2.48 ms
Wall time: 1.42 s


In [17]:
# Returns the commit history
client.query("CALL db.history()")

,commit,nodeCount,edgeCount,partCount
0,d43eace4eb864ff3,0,0,0
1,2103addbce57825e,134984,499500,1


# Query `turingdb` graph

## Use metaqueries to have insight on graph overall structure

<h3>
    To learn more about 📮 Metaqueries, please check TuringDB documentation on this <a href="https://turingdb.mintlify.app/query/cypher_subset#%F0%9F%93%AE-metaqueries">link</a>
</h3>

In [19]:
%%time

# CALL propertyTypes() - returns a column of all the different node and edge properties and their types in the database
command = """
CALL db.propertyTypes()
"""
df_propertyTypes = client.query(command)
if df_propertyTypes.empty:
    print("No result found")
else:
    display(df_propertyTypes)

,id,propertyType,valueType
0,0,Age,Int64
1,1,Date of Admission,String
2,2,Discharge Date,String
3,3,displayName,String
4,4,id,String
5,5,type,String
6,6,Room Number,Int64
7,7,Billing Amount,Double


CPU times: user 3.11 ms, sys: 0 ns, total: 3.11 ms
Wall time: 2.96 ms


In [20]:
# Get node properties
nodes_properties = df_propertyTypes["propertyType"].values.tolist()
print(f"Node properties: {nodes_properties}")

Node properties: ['Age', 'Date of Admission', 'Discharge Date', 'displayName', 'id', 'type', 'Room Number', 'Billing Amount']


In [21]:
%%time

# CALL labels () - returns a column of all the different node labels
command = """
CALL db.labels()
"""
df_labels = client.query(command)
if df_labels.empty:
    print("No result found")
else:
    display(df_labels)

,id,label
0,0,Patient
1,1,Gender
2,2,BloodType
3,3,MedicalCondition
4,4,Doctor
5,5,Hospital
6,6,InsuranceProvider
7,7,AdmissionType
8,8,Medication
9,9,TestResults


CPU times: user 3.21 ms, sys: 0 ns, total: 3.21 ms
Wall time: 3.01 ms


In [22]:
%%time

# CALL edgeTypes() - returns a column of all the different edge types (edge equivalent of node labels)
command = """
CALL db.edgeTypes()
"""
df_edgeTypes = client.query(command)
if df_edgeTypes.empty:
    print("No result found")
else:
    display(df_edgeTypes)

,id,edgeType
0,0,IS
1,1,HAS
2,2,IS_TREATED_BY
3,3,IS_TREATED_IN
4,4,IS_CLIENT_OF
5,5,TYPE
6,6,TOOK_MEDICATION
7,7,HAS_RESULT


CPU times: user 2.88 ms, sys: 0 ns, total: 2.88 ms
Wall time: 2.68 ms


## Counts

In [23]:
%%time

# Find number of nodes and number of edges in the graph
n_nodes = len(client.query("MATCH (n) RETURN n"))
n_edges = len(client.query("MATCH (n)-->(m) RETURN n, m"))
print(f"Graph: {n_nodes:,} nodes and {n_edges:,} edges\n")

Graph: 134,984 nodes and 499,500 edges

CPU times: user 79.4 ms, sys: 0 ns, total: 79.4 ms
Wall time: 88 ms


In [24]:
%%time

# Count all nodes
command = """
MATCH (n)
RETURN COUNT(n)
"""
df_count_nodes = client.query(command)
display(df_count_nodes)

# Count all edges
command = """
MATCH (n)-->()
RETURN COUNT(n)
"""
df_count_edges = client.query(command)
display(df_count_edges)

# Find number of nodes and number of edges in the graph
n_nodes = int(df_count_nodes.loc[0, "COUNT(n)"])
n_edges = int(df_count_edges.loc[0, "COUNT(n)"])
print(f"Graph: {n_nodes:,} nodes and {n_edges:,} edges\n")

,COUNT(n)
0,134984


,COUNT(n)
0,499500


Graph: 134,984 nodes and 499,500 edges

CPU times: user 4.94 ms, sys: 0 ns, total: 4.94 ms
Wall time: 7.77 ms


In [25]:
# Count number of nodes for each label
for label in df_labels["label"]:
    print(100 * '-')
    print(f"label: {label}")
    df_curr_label = client.query(f"""
    MATCH (n:{label})
    RETURN count(n)
    """)
    display(df_curr_label)
    print()
print(100 * '-')

----------------------------------------------------------------------------------------------------
label: Patient


,count(n)
0,55500



----------------------------------------------------------------------------------------------------
label: Gender


,count(n)
0,2



----------------------------------------------------------------------------------------------------
label: BloodType


...


----------------------------------------------------------------------------------------------------
label: Medication


,count(n)
0,5



----------------------------------------------------------------------------------------------------
label: TestResults


,count(n)
0,3



----------------------------------------------------------------------------------------------------


## Queries

In [26]:
%%time

# Match all edges and return them
command = """
MATCH (n)-[e]->(m)
RETURN n.displayName, e, m.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,n.displayName,e,m.displayName
0,Justin RICE,0,Normal
1,Justin RICE,1,Paracetamol
2,Justin RICE,2,Emergency
3,Justin RICE,3,Medicare
4,Justin RICE,4,Cook Inc
...,...,...,...
499495,Kimberly MCLAUGHLIN,499495,Whitney-Franklin
499496,Kimberly MCLAUGHLIN,499496,UnitedHealthcare
499497,Kimberly MCLAUGHLIN,499497,Elective
499498,Kimberly MCLAUGHLIN,499498,Ibuprofen


CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 239 ms


In [27]:
%%time

# Match all edges linking a Patient to an other node
# Return displayName and type properties
command = """
MATCH (n:Patient)-[e]->(m)
RETURN n.type, n.displayName, e, m.type, m.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,n.type,n.displayName,e,m.type,m.displayName
0,Patient,Justin RICE,0,Test Results,Normal
1,Patient,Justin RICE,1,Medication,Paracetamol
2,Patient,Justin RICE,2,Admission Type,Emergency
3,Patient,Justin RICE,3,Insurance Provider,Medicare
4,Patient,Justin RICE,4,Hospital,Cook Inc
...,...,...,...,...,...
499495,Patient,Kimberly MCLAUGHLIN,499495,Hospital,Whitney-Franklin
499496,Patient,Kimberly MCLAUGHLIN,499496,Insurance Provider,UnitedHealthcare
499497,Patient,Kimberly MCLAUGHLIN,499497,Admission Type,Elective
499498,Patient,Kimberly MCLAUGHLIN,499498,Medication,Ibuprofen


CPU times: user 271 ms, sys: 41.4 ms, total: 312 ms
Wall time: 398 ms


In [28]:
%%time

# Find all patients
command = """
MATCH (p:Patient)
RETURN p.id, p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p.id,p.displayName,p.Age
0,28226,Justin RICE,41
1,28225,Brittany LEWIS,56
2,28224,Teresa LYONS,40
3,28223,Leslie VILLANUEVA,53
4,28222,Tammy KEY,80
...,...,...,...
55495,14450,Joseph WELLS,41
55496,13311,Wanda CROSBY,29
55497,14449,Richard BURNS,41
55498,14451,Joan SCOTT,85


CPU times: user 15.2 ms, sys: 8.93 ms, total: 24.1 ms
Wall time: 33.7 ms


In [29]:
%%time

# Find all doctors
command = """
MATCH (d:Doctor)
RETURN d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,d.displayName
0,Joan PETERS
1,Ronnie BATES
2,Kimberly ROSARIO
3,Tyler DILLON
4,James TOWNSEND
...,...
39571,Brett NELSON
39572,Nicholas WHITE
39573,Mark NELSON
39574,Bridget HOWARD


CPU times: user 7.31 ms, sys: 0 ns, total: 7.31 ms
Wall time: 9.21 ms


In [30]:
%%time

# Find all medications
command = """
MATCH (d:Medication)
RETURN d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,d.displayName
0,Ibuprofen
1,Penicillin
2,Paracetamol
3,Lipitor
4,Aspirin


CPU times: user 2.75 ms, sys: 0 ns, total: 2.75 ms
Wall time: 2.71 ms


In [31]:
%%time

# Find patient with specific ID and return all their information
command = """
MATCH (p:Patient)
WHERE p.id = "00000"
RETURN p, p.id, p.displayName, p.type, p.Age, p.`Date of Admission`, p.`Discharge Date`
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p,p.id,p.displayName,p.type,p.Age,p.`Date of Admission`,p.`Discharge Date`
0,49422,00000,Bobby JACKSON,Patient,30,2024-01-31,2024-02-02


CPU times: user 3.43 ms, sys: 949 μs, total: 4.38 ms
Wall time: 12.2 ms


In [32]:
%%time

# Find female patients
command = """
MATCH (p:Patient)-[:IS]->(g:Gender)
WHERE g.displayName = "Female"
RETURN p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p.displayName,p.Age
0,Brittany LEWIS,56
1,Teresa LYONS,40
2,Leslie VILLANUEVA,53
3,Tammy KEY,80
4,Valerie RAMIREZ,51
...,...,...
27721,Peter VELASQUEZ,29
27722,William HARRINGTON,43
27723,Dale LUCAS,25
27724,Joseph WELLS,41


CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 25.9 ms


In [33]:
%%time

# Find patients with Cancer
command = """
MATCH (p:Patient)-[:HAS]->(mc:MedicalCondition)
WHERE mc.displayName = "Cancer"
RETURN p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p.displayName,p.Age
0,Leslie VILLANUEVA,53
1,Tyler CRAWFORD,65
2,Lisa REYES,42
3,Jennifer MCDOWELL,57
4,Joseph BURNS,38
...,...,...
9222,Stephanie ARNOLD,71
9223,Trevor RIOS,66
9224,Robert LITTLE,41
9225,Charles KELLY,48


CPU times: user 7.23 ms, sys: 0 ns, total: 7.23 ms
Wall time: 17.3 ms


In [34]:
%%time

# Find all patients who are treated by a doctor
command = """
MATCH (p:Patient)-[:IS_TREATED_BY]->(d:Doctor)
RETURN p.displayName, d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p.displayName,d.displayName
0,Justin RICE,Andrew BUSH
1,Brittany LEWIS,Laura ZIMMERMAN
2,Teresa LYONS,Jennifer GILMORE
3,Leslie VILLANUEVA,Thomas LOPEZ
4,Tammy KEY,Katrina MUNOZ
...,...,...
55495,Joseph WELLS,Andrew GRAY
55496,Wanda CROSBY,Christopher BALLARD
55497,Richard BURNS,Grace ROBINSON
55498,Joan SCOTT,Jessica BROWNING


CPU times: user 17.8 ms, sys: 417 μs, total: 18.2 ms
Wall time: 32.3 ms


In [35]:
%%time

# Find all patients treated by doctor Kelly OLSON
command = """
MATCH (p:Patient)-[:IS_TREATED_BY]->(d:Doctor)
WHERE d.displayName = "Kelly OLSON"
RETURN p.displayName, d.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p.displayName,d.displayName
0,Edward EDWARDS,Kelly OLSON


CPU times: user 2.78 ms, sys: 1.04 ms, total: 3.82 ms
Wall time: 14.3 ms


In [36]:
%%time

# Find all patients with blood type A+
command = """
MATCH (p:Patient)-[:IS]->(bt:BloodType)
WHERE bt.displayName = "A+"
RETURN p.displayName, p.Age
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,p.displayName,p.Age
0,Meredith SNYDER,62
1,Lawrence LANE,69
2,Travis COLLINS,38
3,Michael CHAN,18
4,Christopher JACOBSON,63
...,...,...
6951,Patricia SHEPARD,83
6952,Stephanie ARNOLD,71
6953,Christina CRAWFORD,19
6954,William HARRINGTON,43


CPU times: user 6.63 ms, sys: 0 ns, total: 6.63 ms
Wall time: 19.9 ms


In [37]:
%%time

# Find all patients who took Paracetamol
command = """
MATCH (p:Patient)-[:TOOK_MEDICATION]->(m:Medication)
WHERE m.displayName = "Paracetamol"
RETURN p.id, p.displayName, m.displayName
"""
df = client.query(command)
df.columns = ["Patient ID", "Patient Name", "Medication"]
display(df)

,Patient ID,Patient Name,Medication
0,28226,Justin RICE,Paracetamol
1,28223,Leslie VILLANUEVA,Paracetamol
2,28216,David MARTIN,Paracetamol
3,28211,Catherine HOFFMAN,Paracetamol
4,28208,Edward SALAZAR,Paracetamol
...,...,...,...
11066,14540,William CHANDLER,Paracetamol
11067,12782,Joseph JOHNSON,Paracetamol
11068,13393,Charles KELLY,Paracetamol
11069,14450,Joseph WELLS,Paracetamol


CPU times: user 8.17 ms, sys: 553 μs, total: 8.72 ms
Wall time: 19.7 ms


# Create subgraph to visualise

In [38]:
import numpy as np

In [39]:
# Get subgraph
subset_nodes = np.unique(df.loc[:100, ["Patient ID", "Medication"]].values).tolist()
subG = G.subgraph(subset_nodes).copy()
print(subG)

# Build CREATE command from subgraph

DiGraph with 102 nodes and 101 edges


In [40]:
subgraph_name = f"{graph_name}_subgraph"
subgraph_name

'healthcare_dataset1_subgraph'

In [41]:
%%time

start_time = time.time()

# Write subgraph to JSONL format
print("Writing subgraph to JSONL...")
jsonl_filename = networkx_to_jsonl(subG, subgraph_name)

# Load JSONL into TuringDB
print(f"\nLoading JSONL into TuringDB as graph '{subgraph_name}' ...")
client.query(f"LOAD JSONL '{jsonl_filename}' AS {subgraph_name}")

execution_time = time.time() - start_time
print(f"\n\u2713 Graph '{subgraph_name}' created in {execution_time:.2f} seconds")

# Set active graph for querying
client.set_graph(subgraph_name)

Writing subgraph to JSONL...
JSONL file written to: /home/ubuntu/.turing/data/healthcare_dataset1_subgraph.jsonl
Graph: 102 nodes, 101 edges

Loading JSONL into TuringDB as graph 'healthcare_dataset1_subgraph' ...

✓ Graph 'healthcare_dataset1_subgraph' created in 0.06 seconds
CPU times: user 2.63 ms, sys: 0 ns, total: 2.63 ms
Wall time: 55.5 ms


<div class="alert alert-block alert-info">
    <h2>
        You can visualise the subgraph directly in the notebook below.
    </h2>
</div>

In [42]:
from pyvis.network import Network

net = Network(
    height="750px",
    width="100%",
    notebook=True,
    bgcolor="#f8f9fa",
    font_color="#212529",
    directed=True,
)

# Node type colors
type_colors = {"Patient": "#3498db", "Medication": "#e74c3c"}

for node, data in subG.nodes(data=True):
    node_type = data.get("type", "Unknown")
    color = type_colors.get(node_type, "#7f8c8d")

    label = data.get("displayName", str(node))

    # Build title based on node type
    if node_type == "Patient":
        title = f"<b>{label}</b><br>Age: {data.get('Age', 'N/A')}<br>Admitted: {data.get('Date of Admission', 'N/A')}<br>Discharged: {data.get('Discharge Date', 'N/A')}"
    else:
        title = f"<b>{label}</b><br>Type: {node_type}"

    net.add_node(node, label=label, color=color, title=title, size=25)

# Edge colors by type
edge_colors = {"took_medication": "#27ae60"}

for source, target, data in subG.edges(data=True):
    edge_type = data.get("type", "")
    color = edge_colors.get(edge_type, "#95a5a6")
    net.add_edge(source, target, title=edge_type, color=color, width=3)

net.toggle_physics(True)
net.show(f"{example_name}_subgraph.html")

healthcare_dataset_subgraph.html


# Use LLM to generate Cypher query

Before running this section, create a `.env` file in the project root with your API keys:

```env
ANTHROPIC_API_KEY=your_key_here
OPENAI_API_KEY=your_key_here
MISTRAL_API_KEY=your_key_here

In [81]:
client.set_graph(graph_name)

In [82]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
if not load_dotenv():
    raise ValueError("LLM credentials not saved in .env file")

In [83]:
api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

In [62]:
"""Build system prompt with TuringDB schema and examples"""

turingdb_cypher_system_prompt = """
You are an expert at converting natural language questions into TuringDB queries.

Your task is to generate syntactically correct TuringDB queries based on natural language input.

VERY IMPORTANT - TuringDB Syntax Guidelines:
1. Return ONLY the TuringDB query, no explanations or markdown formatting
2. Use MATCH, CREATE, WHERE and LIMIT operations only
3. Nodes: (n:Label {property = "value"}) or (n:Label {property: value})
4. Edges: Use DIRECTED syntax with ->
5. Pattern matching: MATCH (n)-[e]->(m)
6. Property matching: Use = operator for exact matching
7. Multiple constraints: (n:Person:Engineer)
8. Return all matched entities: RETURN n, e, m or use RETURN * for all
9. Filter node or edge using WHERE clause. Do not use filters inside the brackets: MATCH (n:Person) WHERE n.name = 'John' RETURN n.firstname, n.lastname

VERY IMPORTANT - FORBIDDEN in TuringDB:
- Do NOT use AS aliases
- Do NOT use SKIP clauses
- Do NOT use WITH clauses
- Do NOT use CALL (except for metaqueries)
- Do NOT use toLower() or other functions

Supported TuringDB Operations:
- MATCH queries: MATCH (n:Label)-[e:Type]->(m) RETURN n, m
- CREATE queries: CREATE (n:Label{property="value"})-[e:Type]->(m:Label)
- Metaqueries: CALL db.propertyTypes(), CALL db.labels(), CALL db.edgeTypes()
- Property types: String ("text" or `text`), Boolean (true/false), Integer (20), Double (20.5)

Examples for few-shot learning:
- Find all persons: MATCH (n:Person) RETURN n
- Find connections: MATCH (n:Person)-[e]->(m:Person) RETURN n, e, m
- Create person: CREATE (n:Person{name="John", age=30})
- Match person with specific name: MATCH (p:Person) WHERE p.name = "John" RETURN p
- Path with 1 hop between Station Paddington and Blackfriars:  MATCH (start:Station{displayName:"Paddington"})-[e1:CONNECTED]->(end:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, end, end.displayName, end.Note
- Path with 2 hops between Station Paddington and Blackfriars: MATCH (start:Station{displayName:"Paddington"})-[e1:CONNECTED]->(s1:Station)-[e2:CONNECTED]->(end:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, s1, s1.displayName, s1.Note, e2.Line, end, end.displayName, end.Note
- Path with 8 hops between Station Paddington and Blackfriars: MATCH (start:Station{displayName:"Paddington"})-[e1:CONNECTED]->(s1:Station)-[e2:CONNECTED]->(s2:Station)-[e3:CONNECTED]->(s3:Station)-[e4:CONNECTED]->(s4:Station)-[e5:CONNECTED]->(s5:Station)-[e6:CONNECTED]->(s6:Station)-[e7:CONNECTED]->(s7:Station)-[e8:CONNECTED]->(end:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, s1, s1.displayName, s1.Note, e2.Line, s2, s2.displayName, s2.Note, e3.Line, s3, s3.displayName, s3.Note, e4.Line, s4, s4.displayName, s4.Note, e5.Line, s5, s5.displayName, s5.Note, e6.Line, s6, s6.displayName, s6.Note, e7.Line, s7, s7.displayName, s7.Note, e8.Line, end, end.displayName, end.Note
- Find all Chinese providers and what they supply: MATCH (n{provider_country:"CHN"}) RETURN n, n.provider_name, n.displayName, n.share_provided, n.type
- Find all deposition tools and their types: MATCH (specific)-[e:IS_TYPE_OF]->(general:Tool_Resource{displayName:"Deposition tools"}) RETURN specific, specific.displayName, specific.provider_name, e, general, general.displayName
"""

In [63]:
# Create system_prompt
system_prompt = f"""
TuringDB Cypher prompt :
{turingdb_cypher_system_prompt}

Here is a sample of nodes from the graph so you know the structure and available properties :
{client.query("MATCH (n) RETURN n LIMIT 3")}

Here is the output of "CALL LABELS ()" command, showing the different node types of the graph :
{client.query("CALL db.labels()")}

Here is also the output of "CALL EDGETYPES ()" command, showing the different edge types of the graph :
{client.query("CALL db.edgeTypes()")}

Very important :
- You MUST follow current TuringDB Syntax Guidelines
- You MUST NOT USE what is FORBIDDEN in TuringDB
- By default, RETURN ALL THE MATCHED NODES AND EDGES AND THEIR PROPERTIES in the RETURN section (except contrary demand from user)
- Use the correct node and edge properties name in the MATCH section.
- Use the correct node and edge properties name in the RETURN section.
- Pay attention to which properties come from nodes or edges, to create a functioning query
- Pay attention to lower and uppercases in properties
- If some properties contain spaces, be careful to wrap them

Give me the query FOLLOWING TURINGDB GUIDELINES AND NOT USING WHAT IS FORBIDDEN for this specific question :
"""


In [114]:
turingdb_cypher_system_prompt = """
You are a TuringDB graph database expert. TuringDB uses a subset of openCypher. Generate ONLY valid TuringDB queries.

Return ONLY the TuringDB query, no explanations or markdown formatting.

## SUPPORTED SYNTAX

### Basic patterns
MATCH (n)                          -- all nodes
MATCH (n:Label)                    -- nodes with label
MATCH (n {prop: 'value'})          -- nodes with property filter
MATCH (a)-[r:REL_TYPE]->(b)        -- directed relationship
MATCH (a)-[r:REL_TYPE]-(b)         -- undirected relationship
MATCH (a)-[r:REL_TYPE*2]->(b)      -- exactly 2 hops
MATCH (a)-[r:REL_TYPE*1..3]->(b)   -- 1 to 3 hops (variable length)

### Return clause
RETURN n                           -- return node
RETURN n.property                  -- return specific property
RETURN n, r, m                     -- return multiple items
RETURN DISTINCT n.property         -- deduplicate results
RETURN n LIMIT 10                  -- return first 10 results

### Filtering
WHERE n.property = 'value'
WHERE n.property <> 'value'
WHERE n.property > 42
WHERE n.property >= 42
WHERE n.property < 42
WHERE n.property <= 42
WHERE n.property IS NULL
WHERE n.property IS NOT NULL
WHERE n.property = 'a' AND m.property = 'b'
WHERE n.property = 'a' OR n.property = 'b'

### Aggregation
RETURN count(n)
RETURN sum(n.amount)
RETURN avg(n.value)
RETURN min(n.value), max(n.value)
RETURN n.category, count(n) ORDER BY count(n) DESC

### Ordering
ORDER BY n.property ASC
ORDER BY n.property DESC

### Schema introspection
CALL db.labels()                   -- list all node labels
CALL db.relationshipTypes()        -- list all relationship types
CALL db.propertyTypes()            -- list all property names and their types

## STRICTLY FORBIDDEN (will cause errors)
- SKIP
- WITH ... AS
- RETURN *
- CREATE, MERGE, DELETE, SET, REMOVE
- UNION
- Subqueries: CALL { ... }
- String functions: toLower(), toUpper(), contains(), startsWith(), endsWith(), trim()
- Math functions: abs(), round(), sqrt()
- List comprehensions: [x IN list WHERE ...]
- Named paths: p = (a)-[*]-(b)
- Reserved variable names: `end`, `start` (use `e`, `s`, or other names)
- Double-quoted strings: use single quotes only — WHERE n.name = 'Alice' NOT "Alice"

## SCHEMA CONTEXT
{schema_context}

## INSTRUCTIONS
1. Use ONLY the labels, relationship types, and property names shown in the schema context above.
2. Always use single quotes for string literals.
3. Always specify relationship direction in MATCH patterns when possible.
4. Do not use any forbidden clauses or functions.
5. If a query cannot be answered within TuringDB's supported syntax, say so explicitly rather than generating an invalid query.
6. For multi-hop traversals, prefer explicit hop counts (e.g., *2) over open-ended ranges when the depth is known.
"""


In [115]:
# Build schema context for LLM
labels_result = client.query("CALL db.labels()")
rel_types_result = client.query("CALL db.edgeTypes()")
prop_types_result = client.query("CALL db.propertyTypes()")
query_sample_nodes = "MATCH (n) RETURN " + ", ".join([f"n.`{prop}`" for prop in prop_types_result["propertyType"].tolist()]) + "LIMIT 3"
sample_nodes = client.query(query_sample_nodes)

schema_context = f"""Node labels: {labels_result}
Relationship types: {rel_types_result}
Property names and types: {prop_types_result}
Sample nodes: {sample_nodes}"""

system_prompt = turingdb_cypher_system_prompt.replace("{schema_context}", schema_context)

In [117]:
# Set natural language query
question = """
Find all patients who took Paracetamol
"""

In [118]:
%%time

provider = "Anthropic"

cypher_query = natural_language_to_cypher(
    question=question,
    system_prompt=system_prompt,
    provider=provider,
    model="claude-haiku-4-5-20251001",
    api_key=api_keys[provider],
)
print(f"cypher_query : {cypher_query}")

cypher_query : MATCH (p:Patient)-[r:TOOK_MEDICATION]->(m:Medication {displayName: 'Paracetamol'})
RETURN p
CPU times: user 5.62 ms, sys: 1.13 ms, total: 6.75 ms
Wall time: 794 ms


In [119]:
df_path = client.query(cypher_query)
if df_path.empty:
    print("--> No result found\n")
else:
    display(df_path)

,p
0,0
1,3
2,10
3,15
4,18
...,...
11066,55476
11067,55486
11068,55493
11069,55495


In [120]:
print("Notebook finished !")

Notebook finished !
